In [1]:
import sys
import os
import csv
from lista_pregoes import *
from obter_url_dos_itens import *
from atualizar_todos_os_dados import *
from juntar_df import *
from atualizar_saldo_vigencia import *

In [7]:
try:    
    url_pregoes_antiga = carregar_lista_pregoes()

except:    
    print('Não há Lista de Pregões salva no computador.')
    url_pregoes_antiga = []


url = 'https://contratos.sistema.gov.br/transparencia/compras?unidade_origem_id=9964&unidade_origem_id_text=160224+-+PARQUE+REGIONAL+DE+MANUTENCAO%2F5&lei=LEI14133&lei_text=LEI14133&persistent-table=true'
url_pregoes_nova = baixar_nova_lista_pregoes(url)

Não há Lista de Pregões salva no computador.
Baixando nova lista de pregões ...
Nova Lista de pregões salva com sucesso!


In [8]:
#Verificando se há novos pregões
    
novos_pregoes = list(set(url_pregoes_nova) - set(url_pregoes_antiga))
novos_pregoes = sorted(novos_pregoes)

print(f'Há {len(novos_pregoes)} novos pregões!')

Há 28 novos pregões!


In [9]:
if not novos_pregoes:
    
    print('Não há novos pregões.')

    #atualizando saldo e vigência
    df_itens_final = atualizar_saldo_vigencia('df_itens_gerenciadora.csv')

    #Salvando o df em arquivo csv
    nome_arquivo = "df_itens_gerenciadora.csv"
    df_itens_final.sort_values(['Número da Compra'], ascending = [False], inplace = True)
    df_itens_final.to_csv(nome_arquivo, sep=';', encoding='utf-8', index=False)        
    print('Arquivo atualizado com sucesso!')
            
else:

    try:

        df_itens_final = pd.read_csv('df_itens_gerenciadora.csv', sep = ';')
        print('Tabela de Dados encontrado no computador!')

        #atualizando saldo e vigência
        #df_itens_final = atualizar_saldo_vigencia('df_itens_gerenciadora.csv')

    except:

        df_itens_final = pd.DataFrame(columns = ['Número da Compra', 'Número do Item', 'Descrição',
       'Descrição Detalhada', 'Início da Vigência', 'Fim da Vigência',
       'Unidade', 'Qtd. Autorizada', 'Fornecedor', 'Val. Unitário',
       'Qtd. Saldo', 'Link do Item', 'Tipo de Compra', 'Número do Pregão',
       'Ano do Pregão', 'UASG', 'Objeto', 'Unidade de Fornecimento'])
    
    for url in novos_pregoes:
        
        #obter a url de cada item do pregão
        url_itens_novos = obter_url_dos_itens([url])

        #obter os dado de cada item 
        df_itens_novos = obter_dados_site_novo(url_itens_novos)
        print('Dados dos itens dos novos pregões obtidos com sucesso!')  
    
        df_itens_novos_site_antigo = obter_unidade_objeto(df_itens_novos)
        print('Objeto e Unidade de Fornecimento dos novos pregões obtidos com sucesso!')

        #tipo_compra = df_itens_novos['Tipo de Compra'][0]
        
        df_itens_merge = merge_df(df_itens_novos,df_itens_novos_site_antigo, tipo_compra = df_itens_novos['Tipo de Compra'][0] )
        print('Dados mesclados com sucesso!')

        df_itens_final = pd.concat([df_itens_final, df_itens_merge])
        print('Tabela dos novos itens adicionada com sucesso!')

        #Salvando o df em arquivo csv
        nome_arquivo = "df_itens_gerenciadora.csv"
        df_itens_final.sort_values(['Número da Compra'], ascending = [False], inplace = True)
        df_itens_final.to_csv(nome_arquivo, sep=';', encoding='utf-8', index=False)        
        print('Arquivo com a tabela dos itens salva com sucesso!')

        #guardando o link dos pregões que já tiveram os dados obtidos
        url_pregoes_antiga.append(url)
        
        # Gravar a lista em um arquivo CSV        
        with open('url_pregoes.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows([[url] for url in url_pregoes_antiga])
            print(f'Dados do Pregão da url {url} obtidos com sucesso! \n')

print('Atualização concluída com sucesso!')

Obtendo dados da url https://contratos.sistema.gov.br/transparencia/compras/372011/itens
Nova Lista de url dos itens salva com sucesso!
Obtendo os dados de cada item...
O Parque/5 possui 18 itens neste pregão.
Dados dos itens dos novos pregões obtidos com sucesso!
Iniciando a obtenção da Unidade de Fornecimento de cada item e o Objeto do Pregão...
Obtendo os dados da url http://comprasnet.gov.br/ConsultaLicitacoes/download/download_editais_detalhe.asp?coduasg=160224&modprp=5&numprp=22023
Dados de Objeto e Unidade de Fornecimento obtidos sucesso. Este Pregão possuía inicialmente 21 itens.
Objeto e Unidade de Fornecimento dos novos pregões obtidos com sucesso!
Dados mesclados com sucesso!
Tabela dos novos itens adicionada com sucesso!
Arquivo com a tabela dos itens salva com sucesso!
Dados do Pregão da url https://contratos.sistema.gov.br/transparencia/compras/372011/itens obtidos com sucesso! 



C:\Users\PQRMNT 5\AppData\Local\Temp\ipykernel_3928\1687771090.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_itens_final = pd.concat([df_itens_final, df_itens_merge])


Obtendo dados da url https://contratos.sistema.gov.br/transparencia/compras/376090/itens
Nova Lista de url dos itens salva com sucesso!
Obtendo os dados de cada item...
O Parque/5 possui 29 itens neste pregão.
Dados dos itens dos novos pregões obtidos com sucesso!
Iniciando a obtenção da Unidade de Fornecimento de cada item e o Objeto do Pregão...
Obtendo os dados da url http://comprasnet.gov.br/ConsultaLicitacoes/download/download_editais_detalhe.asp?coduasg=160224&modprp=5&numprp=32023
Dados de Objeto e Unidade de Fornecimento obtidos sucesso. Este Pregão possuía inicialmente 31 itens.
Objeto e Unidade de Fornecimento dos novos pregões obtidos com sucesso!
Dados mesclados com sucesso!
Tabela dos novos itens adicionada com sucesso!
Arquivo com a tabela dos itens salva com sucesso!
Dados do Pregão da url https://contratos.sistema.gov.br/transparencia/compras/376090/itens obtidos com sucesso! 

Obtendo dados da url https://contratos.sistema.gov.br/transparencia/compras/420699/itens
Nova